In [16]:
from Grafo import Grafo

grafito = Grafo()

grafito.agregar_nodo("pepe")
grafito.agregar_nodo("luis")
grafito.agregar_nodo("solito")
grafito.agregar_arista("pepe","luis",3)
grafito.agregar_arista("luis","pepe",-3)
print(grafito)

grafito.agregar_nodo("amiguis1")
grafito.agregar_nodo("amiguis2")
grafito.agregar_arista("amiguis1","amiguis2",no_dirigido=True)
print(grafito)

{'pepe': 0, 'luis': 1, 'solito': 2}
<Nodos: ['pepe', 'luis', 'solito']
 Adyacencias:
[[ 0.  3.  0.]
 [-3.  0.  0.]
 [ 0.  0.  0.]] >
{'pepe': 0, 'luis': 1, 'solito': 2, 'amiguis1': 3, 'amiguis2': 4}
<Nodos: ['pepe', 'luis', 'solito', 'amiguis1', 'amiguis2']
 Adyacencias:
[[ 0.  3.  0.  0.  0.]
 [-3.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]] >


In [17]:
grafito.eliminar_arista("amiguis1","amiguis2")
print(grafito)

grafito.eliminar_nodo("luis")
print(grafito)

{'pepe': 0, 'luis': 1, 'solito': 2, 'amiguis1': 3, 'amiguis2': 4}
<Nodos: ['pepe', 'luis', 'solito', 'amiguis1', 'amiguis2']
 Adyacencias:
[[ 0.  3.  0.  0.  0.]
 [-3.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]] >
{'pepe': 0, 'solito': 1, 'amiguis1': 2, 'amiguis2': 3}
<Nodos: ['pepe', 'solito', 'amiguis1', 'amiguis2']
 Adyacencias:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]] >


In [18]:
x=['holis','gaston','gato']

if 'pepe' in x:
    print(3)
else:
    print(2)
    
x.index('gaston')

2


1

In [19]:
x

['holis', 'gaston', 'gato']

In [20]:
m={"pepe":1, "gaston":4}
m["gaston"]

4

In [21]:
m["popo"]=-1

In [22]:
m

{'pepe': 1, 'gaston': 4, 'popo': -1}

In [23]:
list(m.values())

[1, 4, -1]

In [24]:
0.0==0

True

In [25]:
ts=grafito._mat_ady[:,0]

In [26]:
ts

<4x1 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Dictionary Of Keys format>

In [27]:
grafito._mat_ady[:,0]=0

In [28]:
grafito._mat_ady.todense()

matrix([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.]])